## Extração de características

Esse notebook está responsável fazer a extraçãop de certas características dos dados, bem como preparar os dados para serem utilizados para treinar o modelo na etapa seguinte.

---

In [1]:
# importando pacotes
import os
import pandas as pd
import numpy as np

import datetime
import holidays

from sklearn.preprocessing import LabelEncoder

import pickle

In [2]:
# agora que temos os dados limpos, podemos seguir com a extralção de caracterpisticas e preparar parte dos dados
# para o treinamento do modelo
pasta_atual = os.getcwd()
pasta_pai = os.path.dirname(pasta_atual)
pasta_data = os.path.join(pasta_pai, "data")

df = pd.read_csv(f"{pasta_data}\\dados_limpos.csv", parse_dates=["date"])\
    .sort_values(by=["date", "appId"])\
    .reset_index(drop=True)

In [3]:
df

,appId,date,dauReal,mauReal,predictionLoss,newinstalls,category,ratings,daily_ratings,reviews,daily_reviews
0,com.app.10626,2024-01-01,2.237000e+05,1.950503e+06,30372.000000,4064.0,FINANCE,418299.0,210.0,111173.0,88.0
1,com.app.11502,2024-01-01,2.399329e+05,1.783888e+06,1786.000000,0.0,NO_CATEGORY,0.0,0.0,0.0,0.0
2,com.app.13071,2024-01-01,2.662241e+05,4.303207e+03,1932.000000,61.0,TRAVEL_AND_LOCAL,1664.0,0.0,927.0,0.0
3,com.app.13421,2024-01-01,6.457805e+05,1.278404e+06,30033.000000,3038.0,FINANCE,423868.0,37.0,46972.0,27.0
4,com.app.13655,2024-01-01,1.014900e+04,7.738700e+04,1569.000000,881.0,FINANCE,62744.0,1.0,26406.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
55180,com.app.97976,2024-10-09,6.087000e+03,6.496978e+04,5688.333333,1158.0,SHOPPING,11313.0,0.0,3704.0,0.0
55181,com.app.97988,2024-10-09,1.917222e+03,2.395267e+04,1660.000000,69.0,SHOPPING,11113.0,0.0,4269.0,0.0
55182,com.app.98198,2024-10-09,1.019312e+04,7.026972e+04,1810.000000,31.0,OTHERS,4074.0,0.0,2244.0,0.0
55183,com.app.98438,2024-10-09,7.054963e+05,2.660798e+06,10678.333333,0.0,NO_CATEGORY,0.0,0.0,0.0,0.0


In [4]:
# Como todos os dados são referentes ao Brasil, vamos adicionar informações de feriados e finais de semana
# do Brasil, pois podem impactar o aesso de alguns aplicativos

feriados_brasil = holidays.Brazil()

def validar_se_dia_trabalho(data):
    ser_dia_semana = data.weekday() < 5
    ser_feriado = data in feriados_brasil

    return ser_dia_semana and not ser_feriado
df["dia_de_trabalho"] = df["date"].apply(validar_se_dia_trabalho)

In [5]:
# vamos também fazer a codificação ciclica das informações de tempo de:
    # semana
    # trimeste
    # ano
    # meia decada

def codificacao_ciclica(aux_df):
    df = aux_df.copy()

    timestamp = df["date"].map(pd.Timestamp.timestamp)

    dia = 24 * 60 * 60
    semana = dia * 7
    ano = 365.2425 * dia
    trimestre = ano / 4
    meia_decada = ano * 5

    df["sin_semana"] = np.sin(timestamp * (2 * np.pi / semana))
    df["cos_semana"] = np.cos(timestamp * (2 * np.pi / semana))

    df["sin_trimestre"] = np.sin(timestamp * (2 * np.pi / trimestre))
    df["cos_trimestre"] = np.cos(timestamp * (2 * np.pi / trimestre))

    df["sin_ano"] = np.sin(timestamp * (2 * np.pi / ano))
    df["cos_ano"] = np.cos(timestamp * (2 * np.pi / ano))

    df["sin_meia_decada"] = np.sin(timestamp * (2 * np.pi / meia_decada))
    df["cos_meia_decada"] = np.cos(timestamp * (2 * np.pi / meia_decada))

    return df

df = codificacao_ciclica(df)

In [6]:
# Também vamos adicionar dados moveis
def adicao_dados_moveis(aux_df, colunas, dias_janela):
    df = aux_df.copy()

    for coluna in colunas:
        df[f"{coluna}_media_{dias_janela}"] = df.groupby("appId")[coluna]\
            .rolling(window = dias_janela, min_periods = 1)\
            .mean()\
            .reset_index(level=0, drop=True)

        df[f"{coluna}_dp_{dias_janela}"] = df.groupby("appId")[coluna]\
            .rolling(window = dias_janela, min_periods = 1)\
            .std()\
            .reset_index(level=0, drop=True)\
            .fillna(0)

    return df

colunas_dados_moveis = ["dauReal", "mauReal", "newinstalls", "ratings", "reviews"] # esses dados são reduntantes com
# daily_ratings e daily_reviews, por isso não estão inclusos
# predictionLoss parecer ser fruto de um outro modelo e, como não tenho conhecimento de como esses dados são populados,
# vou evitar fazer muitas tratavias com esse dado para não complixar desnecessariamente o modelo

df = adicao_dados_moveis(df, colunas = colunas_dados_moveis, dias_janela = 3)
df = adicao_dados_moveis(df, colunas = colunas_dados_moveis, dias_janela = 7)


In [7]:
# Adicionar uma coluna para o dado anterior de dauReal
df["anterior_dauReal"] = df.groupby("appId")["dauReal"].shift(+1)

In [8]:
# Vamos também preparar uma coluna com o dado seguinte de "dauReal"
df["proximo_dauReal"] = df.groupby("appId")["dauReal"].shift(-1)

In [9]:
# converter a coluna de dados categoricos em dados dummy
df = pd.get_dummies(df, columns=['category'])

In [10]:
# Por fim, vamos fazer o LabelEncoding do appId para podermos utilizar o identificador do aplicativo
# como informação relevante para o modelo
le = LabelEncoder()
df["cod_appId"] = le.fit_transform(df["appId"])


In [11]:
df = df.dropna(subset=["proximo_dauReal", "anterior_dauReal"])

In [12]:
pasta_atual = os.getcwd()
pasta_pai = os.path.dirname(pasta_atual)
pasta_data = os.path.join(pasta_pai, "data")

df.to_csv(f"{pasta_data}\\dados_limpos_e_completos.csv", index=False)

with open(f"{pasta_data}\\label_encoder.pkl", 'wb') as f:
    pickle.dump(le, f)
